In [9]:
from retrievers.bm25_retriever import BM25Retriever
from chunkers.fixed_chunker import FixedChunker
from rerankers.none_reranker import NoReranker


corpus = ["Sample document one.", "this is about RAG.", "RAG has a boy system"]
query = "What is RAG?"

chunker = FixedChunker(chunk_size=10)
retriever = BM25Retriever(corpus)
reranker = NoReranker()
retrieved_docs = retriever.retrieve(query, top_k=1)
reranked_docs = reranker.rerank(query, retrieved_docs)

print("Retrieved and Reranked Docs:")
for doc in reranked_docs:
    print("-", doc)

Retrieved and Reranked Docs:
- this is about RAG.
